In [2]:
# Importing necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input

data4 = pd.read_csv("bank-additional-full.csv", sep=";")




In [8]:
  # Ensure consistency across datasets
common_columns = list(set(data4.columns))
combined_data = pd.concat(
    [data4[common_columns]],
    ignore_index=True
)

# Encode categorical variables
categorical_cols = combined_data.select_dtypes(include=['object']).columns
label_encoders = {col: LabelEncoder() for col in categorical_cols}
for col in categorical_cols:
    combined_data[col] = label_encoders[col].fit_transform(combined_data[col])

# Separate features and target
X = combined_data.drop(columns=["y"])
y = combined_data["y"]

# Scale numerical features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)



In [10]:
# ---- Neural Network Training ----
nn_model = Sequential([
    Input(shape=(X_train.shape[1],)),  # Define the input layer with the input shape
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  
])

nn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
nn_model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0)

# Neural Network predictions
nn_preds_train = nn_model.predict(X_train)
nn_preds_test = nn_model.predict(X_test)

# ---- Random Forest Training ----
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Random Forest predictions
rf_preds_train = rf_model.predict_proba(X_train)[:, 1]
rf_preds_test = rf_model.predict_proba(X_test)[:, 1]

# ---- Stacking (Combining Outputs) ----
stack_train = np.column_stack((nn_preds_train, rf_preds_train))
stack_test = np.column_stack((nn_preds_test, rf_preds_test))

meta_model = LogisticRegression()
meta_model.fit(stack_train, y_train)

# Final predictions
final_preds = meta_model.predict(stack_test)

# ---- Evaluation ----
print("Stacked Model Accuracy:", accuracy_score(y_test, final_preds))
print("Classification Report:")
print(classification_report(y_test, final_preds))


901/901 ━━━━━━━━━━━━━━━━━━━━ 1s 750us/step
387/387 ━━━━━━━━━━━━━━━━━━━━ 0s 708us/step
Stacked Model Accuracy: 0.9134094035769199
Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.96      0.95     10968
           1       0.64      0.54      0.58      1389

    accuracy                           0.91     12357
   macro avg       0.79      0.75      0.77     12357
weighted avg       0.91      0.91      0.91     12357



In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

# Load the dataset
df = pd.read_csv('bank-additional-full.csv', sep=';')

# Data Preprocessing

# Handle categorical variables using Label Encoding (for simplicity)
label_encoders = {}
for column in df.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

# Separate features and target variable
X = df.drop('y', axis=1)  # 'y' is the target column
y = df['y']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features (important for neural networks)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 1. Random Forest Model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)

# 2. Neural Network Model
nn_model = Sequential()
nn_model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
nn_model.add(Dense(32, activation='relu'))
nn_model.add(Dense(1, activation='sigmoid'))  # Binary classification

nn_model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
nn_model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)

nn_pred = (nn_model.predict(X_test) > 0.5).astype("int32")

# Evaluation - Random Forest
rf_accuracy = accuracy_score(y_test, rf_pred)
rf_report = classification_report(y_test, rf_pred)

# Evaluation - Neural Network
nn_accuracy = accuracy_score(y_test, nn_pred)
nn_report = classification_report(y_test, nn_pred)

# Print the results
print(f"Random Forest Accuracy: {rf_accuracy}")
print(f"Random Forest Classification Report:\n{rf_report}")
print(f"Neural Network Accuracy: {nn_accuracy}")
print(f"Neural Network Classification Report:\n{nn_report}")


Epoch 1/10


D:\anaconda\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1030/1030 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8762 - loss: 0.2810
Epoch 2/10
1030/1030 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9119 - loss: 0.1959
Epoch 3/10
1030/1030 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9122 - loss: 0.1888
Epoch 4/10
1030/1030 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9166 - loss: 0.1830
Epoch 5/10
1030/1030 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9172 - loss: 0.1799
Epoch 6/10
1030/1030 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9173 - loss: 0.1801
Epoch 7/10
1030/1030 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9197 - loss: 0.1733
Epoch 8/10
1030/1030 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9182 - loss: 0.1741
Epoch 9/10
1030/1030 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9224 - loss: 0.1688
Epoch 10/10
1030/1030 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9202 - loss: 0.1693
258/258 ━━━━━━━━━━━━━━━━━━━━ 0s 825us/step
Random Forest Accuracy: 0.9127215343529983
Random Forest Classification Report:
       